<a href="https://colab.research.google.com/github/pavankumar186/Paper-Implementation-Correspondence-b-w-DeepNN-and-Human-representations/blob/master/Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, zipfile, io
r = requests.get( 'https://github.com/jcpeterson/percept2vec/blob/master/animals.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
!wget https://github.com/jcpeterson/percept2vec/blob/master/turkResults_CogSci2016.mat?raw=true
!mv turkResults_CogSci2016.mat\?raw\=true human_data.mat
from scipy.io import loadmat
human_data = loadmat('human_data.mat')
humMat = human_data['simMatrix']

--2020-07-03 07:35:15--  https://github.com/jcpeterson/percept2vec/blob/master/turkResults_CogSci2016.mat?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jcpeterson/percept2vec/raw/master/turkResults_CogSci2016.mat [following]
--2020-07-03 07:35:15--  https://github.com/jcpeterson/percept2vec/raw/master/turkResults_CogSci2016.mat
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jcpeterson/percept2vec/master/turkResults_CogSci2016.mat [following]
--2020-07-03 07:35:15--  https://raw.githubusercontent.com/jcpeterson/percept2vec/master/turkResults_CogSci2016.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

vgg_model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=True, input_shape = (224, 224, 3))
v_model = keras.Sequential()

for l in vgg_model.layers[:-1]:
  v_model.add(l)

v_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [ ]:
import glob
import re
import numpy as np
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image

data = {}
tag=[]
path = "/content/images/*.png"

for file in glob.glob(path):
    temp1=image.load_img(file,target_size=(224,224))
    temp2 = re.findall(r'\d+', file)
    tag.append(int(temp2[0]))
    data.update({int(temp2[0]) : temp1})

mapping = {}

for i in range(120):
  data[tag[i]] = np.expand_dims(data[tag[i]], axis=0)
  data[tag[i]] = preprocess_input(data[tag[i]])
  feature = v_model.predict(data[tag[i]])
  mapping.update({tag[i] : feature})
  mapping[tag[i]] = np.reshape(mapping[tag[i]],4096)



In [ ]:
tag.sort()

li = []

for i in range(120):
  li.append(mapping[tag[i]])

in_mat = np.asarray(li)
in_mat = np.reshape(in_mat, (120,4096))
print(in_mat)

Mat = in_mat.dot(in_mat.transpose())
simMatOrig = Mat
Mat = np.reshape(Mat, (14400))
print(Mat)



inp= []
for i in range(120):
  for j in range(120):
    inp2.append(F[i]*F[j])

inp2 = np.asarray(inp2)
print(inp2.shape )

[[0.         1.5838431  0.         ... 0.         0.47101927 0.        ]
 [0.7050711  0.         1.9185705  ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.01230419]
 ...
 [0.         5.9127965  0.         ... 0.         0.         0.        ]
 [0.         0.         2.6115813  ... 0.         0.         0.        ]
 [0.         1.7740858  0.         ... 0.         0.         0.5558691 ]]
[7379.0215  3241.447   1205.5062  ... 1785.7758   408.06677 3472.9753 ]


In [ ]:
humMat=np.reshape(humMat,(120,120))

humanSimMat = humMat
uniMat=[]
for i in range(120):
  for j in range(i,120):
    uniMat.append(humMat[i,j])
uniMat=np.asarray(uniMat)
print(uniMat.shape)
humMat = humMat.reshape((14400))

cor_mat = np.corrcoef(Mat, humMat)
r = cor_mat[0,1]

r2 = r**2
print(r2)


(7260,)
0.44783041226328657


In [ ]:
# normal usage
inp = []
for i in range(120):
  for j in range(i,120):
    inp.append(F[i]*F[j])

inp = np.asarray(inp)
print(inp.shape )

(7260, 4096)


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
clf2 = Ridge(solver='sag', fit_intercept=False)
parameters = {'alpha': [0.01, 1, 10,100,1000,1e4, 50000, 1e5,1e6]}
ridge_regressor = GridSearchCV(clf2, parameters, scoring='neg_mean_squared_error', cv=6)
ridge_regressor.fit(inp, uniqueSim)
best_clf = ridge_regressor.best_estimator_
print(best_clf)
# best_clf.fit(inp, uniqueSim)

In [ ]:
a=cross_validate(best_clf,inp2,actualSimMat,scoring="r2",cv=6)

In [ ]:
a['test_score'].mean()

In [ ]:
simPredict = best_clf.predict(inp2)

correlation_matrix = np.corrcoef(simPredict, actualSimMat)
correlation_xy = correlation_matrix[0,1]

r_squared = correlation_xy**2
print(r_squared)

In [ ]:
def dissimilarity(S):
  D = np.zeros_like(S)
  for i in np.arange(D.shape[0]):
    for j in np.arange(D.shape[1]):
      D[i][j] = np.sqrt(abs(S[i][i] + S[j][j] - 2*S[i][j]))

  return D

In [ ]:
from sklearn.manifold import MDS
visual = MDS(n_components = 2, dissimilarity = 'precomputed', random_state = 1)
coord = visual.fit_transform(dissimilarity(humanSimMat))
plt.scatter(coord[:,0], coord[:,1])
plt.title("Original Data MDS Plot")

In [ ]:
visual = MDS(n_components = 2, dissimilarity = 'precomputed', random_state = 1)
coord = visual.fit_transform(dissimilarity(simMatOrig))
plt.scatter(coord[:,0], coord[:,1])
plt.title("Raw MDS Plot")

In [ ]:
simMatMod = simPredict.reshape((120,120))
coord = visual.fit_transform(dissimilarity(simMatMod))
plt.scatter(coord[:,0], coord[:,1])
plt.title("Adjusted MDS Plot")

Human Similarity Hierarchical Clustering:

In [ ]:
import plotly.figure_factory as ff
import numpy as np

 # 15 samples, with 12 dimensions each
fig = ff.create_dendrogram(humanSimMat, color_threshold=10)
fig.update_layout(width=800, height=500)
fig.show()

Adjusted Similarity Hierarchical Clustering:


In [ ]:
fig = ff.create_dendrogram(simMatMod, color_threshold=15)
fig.update_layout(width=800, height=500)
fig.show()

Raw Similarity Hierarchical Clustering:


In [ ]:
fig = ff.create_dendrogram(simMatOrig, color_threshold=3000)
fig.update_layout(width=800, height=500)
fig.show()